# Alloys data preprocessing

**Initial edit:** *Lukas Vlcek, 2019-03-06*

* Data from Michel Gao's paper SI (ref: )

* Read and clean data
* Store for future processing in a CSV file

## Setup

In [1]:
# basic libraries
import os
import numpy as np
import pandas as pd

In [2]:
# project directories
data_raw = '../data/raw'
data_interim = '../data/interim'
data_processed = '../data/processed'
reports_dir = '../reports'
src_dir = '../src'

## Read data

In [3]:
# import CSV (which was exported from Excel), in need for a serious cleanup
df = pd.read_csv(os.path.join(data_interim, 'Gao_from_xls.csv'))

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 46 columns):
Alloy             360 non-null object
Unnamed: 1        0 non-null float64
Unnamed: 2        0 non-null float64
Unnamed: 3        0 non-null float64
Unnamed: 4        0 non-null float64
Unnamed: 5        0 non-null float64
Unnamed: 6        0 non-null float64
Unnamed: 7        0 non-null float64
Unnamed: 8        0 non-null float64
Unnamed: 9        0 non-null float64
Unnamed: 10       0 non-null float64
Unnamed: 11       0 non-null float64
Unnamed: 12       0 non-null float64
Unnamed: 13       0 non-null float64
Unnamed: 14       0 non-null float64
Unnamed: 15       0 non-null float64
                 355 non-null float64
Unnamed: 17       0 non-null float64
Unnamed: 18       0 non-null float64
Unnamed: 19       0 non-null float64
ΔHmix             355 non-null float64
Unnamed: 21       0 non-null float64
Unnamed: 22       0 non-null float64
Unnamed: 23       0 non-null float64
ΔS

In [5]:
# Drop all columns with all values NaN
df = df.dropna(axis='columns', how='all')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 17 columns):
Alloy             360 non-null object
                 355 non-null float64
ΔHmix             355 non-null float64
ΔSmix             355 non-null float64
Tm                355 non-null float64
Unnamed: 28       1 non-null float64
Ω                 355 non-null object
              355 non-null object
Δ                355 non-null float64
VEC               355 non-null float64
                 355 non-null float64
RMS [%]           355 non-null float64
ΔHImmix
/ΔHmix    314 non-null float64
kcr               355 non-null float64
Unnamed: 43       41 non-null float64
                 314 non-null float64
E2/E0 [x104]      355 non-null float64
dtypes: float64(14), object(3)
memory usage: 48.0+ KB


In [7]:
# rename columns
column_names = ['alloy', 'density', 'DH_mix', 'DS_mix', 'T_m', 'xxx_1', 'Omega', 'delta', 'D_elneg', 'VEC']
column_names += ['phi', 'RMS', 'DH_im', 'k_cr', 'xxx_2', 'eta', 'E2_E0']
df.columns = column_names
df.head()

,alloy,density,DH_mix,DS_mix,T_m,xxx_1,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,xxx_2,eta,E2_E0
0,FCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ag1Au1 [12],14.88,-6.0,5.76,1286.13,NaN,1.24,0.35,0.31,11.0,3.146300e+02,0.35,1.370000,1.270000,NaN,0.50,0.06
2,Co1Fe1 [12],8.37,-1.0,5.76,1789.50,NaN,10.31,0.79,0.03,8.5,2.837100e+02,0.79,5.790000,3.270000,NaN,0.98,0.31
3,Co1Ni1 [12],8.90,0.0,5.76,1748.00,NaN,200**,0,0.02,9.5,5.200000e+14,0.00,inf,inf,NaN,2.73,0.00
4,Cu1Ni1 [12],8.91,4.0,5.76,1542.88,NaN,2.22,1.19,0.01,10.5,7.744000e+01,1.19,-0.145000,1.490000,NaN,8.45,0.70


## Cleanup

In [8]:
# fix xxx columns (after checking with the excel file and the original paper SI)

# 1. The single non-null value in xxx_1 is an error. It should be a decimal part of the number in T_m.
# Also, both T_m and xxx_1 are on a wrong row, should be moved to the previous one
df.loc[pd.notna(df['xxx_1'])]
#this is how the row looks

,alloy,density,DH_mix,DS_mix,T_m,xxx_1,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,xxx_2,eta,E2_E0
156,NaN,NaN,NaN,NaN,1885.0,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df[154:160]

,alloy,density,DH_mix,DS_mix,T_m,xxx_1,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,xxx_2,eta,E2_E0
154,Co1Cr1Fe1Ni1Ti0.5 [79],7.58,-11.56,13.15,1879.44,NaN,2.14,4.93,0.13,7.78,9.36,4.86,1.68,1.47,NaN,0.32,7.61
155,"Co1Cr1Fe1Ni1Ti1 [51, 80]",7.15,-16.32,13.38,NaN,NaN,1.55,6.13,0.14,7.40,4.16,6.08,1.72,1.34,NaN,0.33,13.16
156,NaN,NaN,NaN,NaN,1885.00,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,Co1Cr1Fe1Ni1V1 [51],7.69,-8.96,13.38,1934.00,NaN,2.89,2.88,0.12,7.60,35.59,2.87,1.59,1.64,NaN,0.59,2.91
158,Co1Cr1Fe1Ni1W1 [51],11.02,-2.88,13.38,2236.40,NaN,10.39,4.65,0.23,7.80,18.61,4.62,5.03,3.29,NaN,0.31,7.58
159,Co1Cr1Fe1Ni1Y1 [51],6.62,-9.28,13.38,1857.20,NaN,2.68,15.72,0.26,7.20,1.12,15.60,1.28,1.59,NaN,0.32,86.45


In [10]:
df.loc[155, 'T_m'] = df.loc[156, 'T_m'] + df.loc[156, 'xxx_1']

In [11]:
df = df.drop(156)
df[154:160]

,alloy,density,DH_mix,DS_mix,T_m,xxx_1,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,xxx_2,eta,E2_E0
154,Co1Cr1Fe1Ni1Ti0.5 [79],7.58,-11.56,13.15,1879.44,NaN,2.14,4.93,0.13,7.78,9.360,4.86,1.680,1.47,NaN,0.32,7.61
155,"Co1Cr1Fe1Ni1Ti1 [51, 80]",7.15,-16.32,13.38,1885.60,NaN,1.55,6.13,0.14,7.40,4.160,6.08,1.720,1.34,NaN,0.33,13.16
157,Co1Cr1Fe1Ni1V1 [51],7.69,-8.96,13.38,1934.00,NaN,2.89,2.88,0.12,7.60,35.590,2.87,1.590,1.64,NaN,0.59,2.91
158,Co1Cr1Fe1Ni1W1 [51],11.02,-2.88,13.38,2236.40,NaN,10.39,4.65,0.23,7.80,18.610,4.62,5.030,3.29,NaN,0.31,7.58
159,Co1Cr1Fe1Ni1Y1 [51],6.62,-9.28,13.38,1857.20,NaN,2.68,15.72,0.26,7.20,1.120,15.60,1.280,1.59,NaN,0.32,86.45
160,Co1Cr1Fe1Ni1Zr1 [51],7.61,-22.72,13.38,1923.00,NaN,1.13,10.19,0.21,7.40,0.494,10.10,0.977,1.25,NaN,0.32,36.33


In [12]:
# 2. non-null values in xxx_2 are misplaced and should be moved to eta
df = df.assign(eta=lambda x:x['eta'].fillna(0) + x['xxx_2'].fillna(0))
df.loc[pd.notna(df['xxx_2'])].head()

,alloy,density,DH_mix,DS_mix,T_m,xxx_1,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,xxx_2,eta,E2_E0
26,Mo1Nb1 [12],9.36,-6.0,5.76,2823.0,NaN,2.71,2.44,0.28,5.5,21.04,2.44,2.140000,1.600000,0.70,0.70,2.97
27,Mo1Ta1 [12],13.71,-5.0,5.76,3093.0,NaN,3.57,2.44,0.33,5.5,23.98,2.44,3.720000,1.780000,0.53,0.53,2.97
28,Mo1Ti1 [12],7.21,-4.0,5.76,2418.5,NaN,3.48,2.1,0.31,5.0,32.10,2.10,4.030000,1.770000,0.48,0.48,2.20
29,Mo1V1 [12],8.31,0.0,5.76,2539.5,NaN,200**,1.82,0.27,5.5,59.92,1.82,inf,inf,0.66,0.66,1.65
30,Mo1W1 [12],14.82,0.0,5.76,3295.5,NaN,200**,0.36,0.10,6.0,1562.29,0.36,inf,inf,13.53,13.53,0.06


In [13]:
df = df.drop(columns=['xxx_1', 'xxx_2'])
df.head()

,alloy,density,DH_mix,DS_mix,T_m,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,eta,E2_E0
0,FCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
1,Ag1Au1 [12],14.88,-6.0,5.76,1286.13,1.24,0.35,0.31,11.0,3.146300e+02,0.35,1.370000,1.270000,0.50,0.06
2,Co1Fe1 [12],8.37,-1.0,5.76,1789.50,10.31,0.79,0.03,8.5,2.837100e+02,0.79,5.790000,3.270000,0.98,0.31
3,Co1Ni1 [12],8.90,0.0,5.76,1748.00,200**,0,0.02,9.5,5.200000e+14,0.00,inf,inf,2.73,0.00
4,Cu1Ni1 [12],8.91,4.0,5.76,1542.88,2.22,1.19,0.01,10.5,7.744000e+01,1.19,-0.145000,1.490000,8.45,0.70


In [14]:
df.reset_index(drop=True, inplace=True)

In [15]:
# Remove rows with lattice type information and assign the lattice info to individual alloys
df.loc[pd.isna(df.density)]

,alloy,density,DH_mix,DS_mix,T_m,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,eta,E2_E0
0,FCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
25,BCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
67,HCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
82,multi-phase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
300,amorphous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [16]:
# get indices of the starting rows and the corresponding lattice designations
indices = df.loc[pd.isna(df.density)].index
lattices = list(df.loc[indices]['alloy'])
indxs = list(indices)
indxs.append(len(df))
print(indxs)

[0, 25, 67, 82, 300, 360]


In [17]:
# prepare the lattice column with a default value
df = df.assign(lattice='unknown')

In [18]:
for i, lat in enumerate(lattices):
    df.loc[indxs[i]:indxs[i+1], 'lattice'] = lat

In [19]:
df = df.drop(indices)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 1 to 359
Data columns (total 16 columns):
alloy      355 non-null object
density    355 non-null float64
DH_mix     355 non-null float64
DS_mix     355 non-null float64
T_m        355 non-null float64
Omega      355 non-null object
delta      355 non-null object
D_elneg    355 non-null float64
VEC        355 non-null float64
phi        355 non-null float64
RMS        355 non-null float64
DH_im      314 non-null float64
k_cr       355 non-null float64
eta        355 non-null float64
E2_E0      355 non-null float64
lattice    355 non-null object
dtypes: float64(12), object(4)
memory usage: 47.1+ KB


In [21]:
# there are '[]' characters in the delta column, preventing it to be converted to a numerical type - eliminate it 
df.loc[df.delta.str.contains('\[')]

,alloy,density,DH_mix,DS_mix,T_m,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,eta,E2_E0,lattice
243,Al1Cr1Cu1Fe1Ni1Ti1 [98],6.26,-13.67,14.9,1658.54,1.81,6.[57],0.15,7.0,5.71,6.27,2.49,1.4,0.21,13.19,multi-phase


In [22]:
df.loc[243, 'delta'] = '6.57'

In [23]:
# cast delta column as float
df.delta = df.delta.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 1 to 359
Data columns (total 16 columns):
alloy      355 non-null object
density    355 non-null float64
DH_mix     355 non-null float64
DS_mix     355 non-null float64
T_m        355 non-null float64
Omega      355 non-null object
delta      355 non-null float64
D_elneg    355 non-null float64
VEC        355 non-null float64
phi        355 non-null float64
RMS        355 non-null float64
DH_im      314 non-null float64
k_cr       355 non-null float64
eta        355 non-null float64
E2_E0      355 non-null float64
lattice    355 non-null object
dtypes: float64(13), object(3)
memory usage: 57.1+ KB


In [24]:
# Column Omega contains special values 200** , which should be infinity. First convert to NaN then to Inf
df.Omega = pd.to_numeric(df.Omega, errors='coerce').fillna(float('inf'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 1 to 359
Data columns (total 16 columns):
alloy      355 non-null object
density    355 non-null float64
DH_mix     355 non-null float64
DS_mix     355 non-null float64
T_m        355 non-null float64
Omega      355 non-null float64
delta      355 non-null float64
D_elneg    355 non-null float64
VEC        355 non-null float64
phi        355 non-null float64
RMS        355 non-null float64
DH_im      314 non-null float64
k_cr       355 non-null float64
eta        355 non-null float64
E2_E0      355 non-null float64
lattice    355 non-null object
dtypes: float64(14), object(2)
memory usage: 57.1+ KB


In [25]:
# Some omegas are NaN, but there are quite a few Infs in various columns
#df.loc[pd.isna(df.DH_im)]

In [26]:
# split Alloy names into a new formula column and a reference column
new_cols = df.apply(lambda x: [x.alloy.split()[0], ' '.join(x.alloy.split()[1:])], axis=1, result_type='expand')
new_cols.head()

,0,1
1,Ag1Au1,[12]
2,Co1Fe1,[12]
3,Co1Ni1,[12]
4,Cu1Ni1,[12]
5,Fe1Ni1,[12]


In [27]:
df = df.assign(formula=new_cols[0], refs=new_cols[1])
df = df.drop(columns=['alloy'])
df.head()

,density,DH_mix,DS_mix,T_m,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,eta,E2_E0,lattice,formula,refs
1,14.88,-6.0,5.76,1286.13,1.240000,0.35,0.31,11.0,3.146300e+02,0.35,1.370000,1.270000,0.50,0.06,FCC,Ag1Au1,[12]
2,8.37,-1.0,5.76,1789.50,10.310000,0.79,0.03,8.5,2.837100e+02,0.79,5.790000,3.270000,0.98,0.31,FCC,Co1Fe1,[12]
3,8.90,0.0,5.76,1748.00,inf,0.00,0.02,9.5,5.200000e+14,0.00,inf,inf,2.73,0.00,FCC,Co1Ni1,[12]
4,8.91,4.0,5.76,1542.88,2.220000,1.19,0.01,10.5,7.744000e+01,1.19,-0.145000,1.490000,8.45,0.70,FCC,Cu1Ni1,[12]
5,8.37,-2.0,5.76,1769.50,5.100000,0.79,0.04,9.0,2.525500e+02,0.79,4.680000,2.120000,0.60,0.31,FCC,Fe1Ni1,[12]


In [28]:
# place formula as the first column and lattice as second
new_order = list(df.columns)
new_order = new_order[-2:-1] + new_order[-3:-2] + new_order[:-3] + new_order[-1:]
assert len(df.columns) == len(new_order), "Incorrect number of columns"
df = df[new_order]
df.head()

,formula,lattice,density,DH_mix,DS_mix,T_m,Omega,delta,D_elneg,VEC,phi,RMS,DH_im,k_cr,eta,E2_E0,refs
1,Ag1Au1,FCC,14.88,-6.0,5.76,1286.13,1.240000,0.35,0.31,11.0,3.146300e+02,0.35,1.370000,1.270000,0.50,0.06,[12]
2,Co1Fe1,FCC,8.37,-1.0,5.76,1789.50,10.310000,0.79,0.03,8.5,2.837100e+02,0.79,5.790000,3.270000,0.98,0.31,[12]
3,Co1Ni1,FCC,8.90,0.0,5.76,1748.00,inf,0.00,0.02,9.5,5.200000e+14,0.00,inf,inf,2.73,0.00,[12]
4,Cu1Ni1,FCC,8.91,4.0,5.76,1542.88,2.220000,1.19,0.01,10.5,7.744000e+01,1.19,-0.145000,1.490000,8.45,0.70,[12]
5,Fe1Ni1,FCC,8.37,-2.0,5.76,1769.50,5.100000,0.79,0.04,9.0,2.525500e+02,0.79,4.680000,2.120000,0.60,0.31,[12]


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 1 to 359
Data columns (total 17 columns):
formula    355 non-null object
lattice    355 non-null object
density    355 non-null float64
DH_mix     355 non-null float64
DS_mix     355 non-null float64
T_m        355 non-null float64
Omega      355 non-null float64
delta      355 non-null float64
D_elneg    355 non-null float64
VEC        355 non-null float64
phi        355 non-null float64
RMS        355 non-null float64
DH_im      314 non-null float64
k_cr       355 non-null float64
eta        355 non-null float64
E2_E0      355 non-null float64
refs       355 non-null object
dtypes: float64(14), object(3)
memory usage: 49.9+ KB


In [30]:
# save to a CSV without the index numbers
df.to_csv(os.path.join(data_interim, 'alloys_clean.csv'), index=False)